In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

In [ ]:
# mount google MyDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set data paths
d_train='/content/drive/MyDrive/tmp_sat/data_train'
d_val='/content/drive/MyDrive/tmp_sat/data_val'
d_test='/content/drive/MyDrive/tmp_sat/data_test'

In [ ]:

# 1. Data Preparation and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    d_train,
    target_size=(150, 150),  # Adjust target size as needed
    batch_size=30,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    d_val,
    target_size=(150, 150),
    batch_size=30,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    d_test,
    target_size=(150, 150),
    batch_size=30,
    class_mode='categorical'
)



In [ ]:

# 2. Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='sigmoid')  # 3-class classification
])

In [ ]:
# 3. Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy'
              metrics=['accuracy'])


In [ ]:
# 4. Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


In [ ]:
# 5. Evaluate the Model (Optional)
loss, accuracy = model.evaluate(validation_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [ ]:
# 5. Evaluate the Model (Optional)
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [ ]:
# 6. Save the model (Optional)
model.save('satellite_classifier.h5') #saves the model.


In [ ]:
# 7. Load model for predictions (Optional)
# loaded_model = tf.keras.models.load_model('satellite_classifier.h5')
# predictions = loaded_model.predict(validation_generator)